In [ ]:
!pip install rdkit


In [2]:
import pandas as pd
import numpy as np
import rdkit as rd
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import Descriptors

In [ ]:
df_CB2 = pd.read_csv('CB2_initial_file.csv', delimiter=";")
df_CKR5 = pd.read_csv('CKR5_initial_file.csv', delimiter=";")
df_DRD1 = pd.read_csv('DRD1_initial_file.csv', delimiter=";")
df_B2AR = pd.read_csv('b2ar_initial_file.csv', delimiter=";")

names = ['cb2_mol_des.csv', 'ckr5_mol_des.csv', 'drd1_mol_des.csv', 'b2ar_mol_des.csv']
datasets = [df_CB2, df_CKR5, df_DRD1, df_B2AR]
new_datasets = []

counter = 0

for df in datasets:

  # Начальная очитска датасета от лишней информации
  df_new = df[['Molecule ChEMBL ID', 'Smiles', 'Target Name', 'Standard Value' ]]
  df_new.dropna(how = 'any', inplace=True)
  df_new.reset_index(drop=True, inplace=True)
  df_new.rename(columns = {"Molecule ChEMBL ID" : "ID", "Standard Value" : "Ki"}, inplace=True)

  descriptor_names = list(rdMolDescriptors.Properties.GetAvailableProperties())
  get_descriptors = rdMolDescriptors.Properties(descriptor_names)
  num_descriptors = len(descriptor_names)

  # Инициализация пустой матрицы для дескрипторов
  descriptors_set = np.empty((0, num_descriptors), float)

  # Создание сета дескрипторов
  for _, row in df_new.iterrows():
    smiles = row['Smiles']
    molecule = Chem.MolFromSmiles(smiles)

    if molecule is not None:
      descriptors = np.array(get_descriptors.ComputeProperties(molecule)).reshape((-1, num_descriptors))
      descriptors_set = np.append(descriptors_set, descriptors, axis=0)

  # Создание DataFrame с дескрипторами
  df_descriptors = pd.DataFrame(descriptors_set, columns=descriptor_names)
  df_descriptors.reset_index(drop=True, inplace=True)

  # Объединение существующего датасета с новыми дескрипторами
  df_clean = pd.concat([df_new, df_descriptors], axis=1)

  df_clean.dropna(how = 'any', inplace=True)

  df_clean.to_csv(names[counter], index=False)

  new_datasets.append(df_clean)

  counter += 1